In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [2]:
articles = pd.read_csv('Gaza_textcontain_before_new.csv')
authors = pd.read_csv('Gaza_textcontain_before_new_authors.csv')

In [4]:
# date published
#remoe where published time = published time not found

articles = articles.drop(articles[articles['published_time'] == 'Published time not found'].index)

articles['published_time'] = articles['published_time'].str[11:]
articles['published_time'] = articles['published_time'].str[:12]
#to date mmm dd, yyyy
articles['published_time'] = pd.to_datetime(articles['published_time'], format='%b %d, %Y')
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Various celebrations held across world to cele...,By Xinhua,2023-09-29,Australian youths sing Chinese version of the ...,PHOTO,WORLD,Australian youths sing Chinese version of the ...
1,First Saudi ambassador to Palestine arrives in...,By Xinhua,2023-09-27,Saudi ambassador to Palestine Naif bin Bandar ...,PHOTO,WORLD,Saudi ambassador to Palestine Naif bin Bandar ...
2,China releases a white paper on its proposals ...,By GT staff reporters,2023-09-26,Photo: Li Xuanmin/GT The Chinese government re...,CHINA,POLITICS,Photo: Li Xuanmin/GT
3,"Kaparot ceremony in Tel Aviv, Israel",By Xinhua,2023-09-22,Ultra Orthodox Jews perform the Kaparot ceremo...,PHOTO,WORLD,Ultra Orthodox Jews perform the Kaparot ceremo...
4,"1,100 Gazan kidney patients facing drug shorta...",By Xinhua,2023-09-21,A young Palestinian patient receives treatment...,PHOTO,WORLD,A young Palestinian patient receives treatment...
...,...,...,...,...,...,...,...
477,F-35: Closer Look at Colossally Costly Jet Tha...,By Sputnik,2023-09-21,A US Marine Corps F-35B went down over South C...,WORLD,AMERICAS,Picture description not found
478,Toxic explosion in Nebraska shows US unable to...,By Xu Yelu,2023-09-15,A resident of East Palestine in the state of O...,WORLD,AMERICAS,A resident of East Palestine in the state of O...
479,Palestine sends rescue team to floods-hit east...,By Xinhua,2023-09-14,Rescuers of a Palestinian rescue team prepare ...,PHOTO,WORLD,Rescuers of a Palestinian rescue team prepare ...
480,Armed clashes in Palestinian refugee camp inte...,By Xinhua,2023-09-14,Palestinians from Ain Al-Helweh camp prepare t...,PHOTO,WORLD,Palestinians from Ain Al-Helweh camp prepare t...


In [5]:
#stats published time
articles['author']

0                  By Xinhua
1                  By Xinhua
2      By GT staff reporters
3                  By Xinhua
4                  By Xinhua
               ...          
477               By Sputnik
478               By Xu Yelu
479                By Xinhua
480                By Xinhua
481                Xu Liuliu
Name: author, Length: 482, dtype: object

In [6]:
# celan authors
articles['author'] = articles['author'].apply(lambda x: x[3:] if x.startswith("By ") else x)
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Various celebrations held across world to cele...,Xinhua,2023-09-29,Australian youths sing Chinese version of the ...,PHOTO,WORLD,Australian youths sing Chinese version of the ...
1,First Saudi ambassador to Palestine arrives in...,Xinhua,2023-09-27,Saudi ambassador to Palestine Naif bin Bandar ...,PHOTO,WORLD,Saudi ambassador to Palestine Naif bin Bandar ...
2,China releases a white paper on its proposals ...,GT staff reporters,2023-09-26,Photo: Li Xuanmin/GT The Chinese government re...,CHINA,POLITICS,Photo: Li Xuanmin/GT
3,"Kaparot ceremony in Tel Aviv, Israel",Xinhua,2023-09-22,Ultra Orthodox Jews perform the Kaparot ceremo...,PHOTO,WORLD,Ultra Orthodox Jews perform the Kaparot ceremo...
4,"1,100 Gazan kidney patients facing drug shorta...",Xinhua,2023-09-21,A young Palestinian patient receives treatment...,PHOTO,WORLD,A young Palestinian patient receives treatment...
...,...,...,...,...,...,...,...
477,F-35: Closer Look at Colossally Costly Jet Tha...,Sputnik,2023-09-21,A US Marine Corps F-35B went down over South C...,WORLD,AMERICAS,Picture description not found
478,Toxic explosion in Nebraska shows US unable to...,Xu Yelu,2023-09-15,A resident of East Palestine in the state of O...,WORLD,AMERICAS,A resident of East Palestine in the state of O...
479,Palestine sends rescue team to floods-hit east...,Xinhua,2023-09-14,Rescuers of a Palestinian rescue team prepare ...,PHOTO,WORLD,Rescuers of a Palestinian rescue team prepare ...
480,Armed clashes in Palestinian refugee camp inte...,Xinhua,2023-09-14,Palestinians from Ain Al-Helweh camp prepare t...,PHOTO,WORLD,Palestinians from Ain Al-Helweh camp prepare t...


In [40]:
[articles['author'].unique(), authors['author_name'].unique()]

[array(['Xinhua', 'GT staff reporters', 'Sputnik', 'Xu Yelu', 'Xu Liuliu',
        'Yang Sheng', 'Global Times', 'Lou Kang', 'VCG', 'Deng Xiaoci',
        'Wan Hengyi', 'WAM', 'Yin Yeping', 'Long Xingchun and Wang Geyan',
        'Mark Blacklock', 'Qi Xijia', 'Liu Xin', 'Maitreya Bhakal',
        'Ding Long', 'Lu Wenao', 'Zhao Xiaozhuo', 'Wang Qi',
        'Bradley Blankenship', 'Anthony Moretti', 'Kishore Mahbubani',
        'Xie Wenting and Bai Yunyi', 'Fan Anqi', 'Author not found',
        'Zhang Hua', 'Bai Yunyi', 'Ebrahim Hashem', 'Dakotah Lilly',
        'Xin Ping', 'IC'], dtype=object),
 array(['Xu Liuliu', 'Yang Sheng', 'Deng Xiaoci', 'Liu Xin', 'Wang Qi',
        'Bradley Blankenship'], dtype=object)]

In [7]:
articles['article_text'] = articles.apply(lambda x: x['article_text'].replace(x['picture_description'], ''), axis=1)
articles

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Various celebrations held across world to cele...,Xinhua,2023-09-29,Musicians perform during a Mid-Autumn Festiva...,PHOTO,WORLD,Australian youths sing Chinese version of the ...
1,First Saudi ambassador to Palestine arrives in...,Xinhua,2023-09-27,\n\nSaudi ambassador to Palestine Naif bin Ba...,PHOTO,WORLD,Saudi ambassador to Palestine Naif bin Bandar ...
2,China releases a white paper on its proposals ...,GT staff reporters,2023-09-26,The Chinese government released a white paper...,CHINA,POLITICS,Photo: Li Xuanmin/GT
3,"Kaparot ceremony in Tel Aviv, Israel",Xinhua,2023-09-22,\n\nUltra Orthodox Jews perform the Kaparot c...,PHOTO,WORLD,Ultra Orthodox Jews perform the Kaparot ceremo...
4,"1,100 Gazan kidney patients facing drug shorta...",Xinhua,2023-09-21,\n\nA young Palestinian patient receives trea...,PHOTO,WORLD,A young Palestinian patient receives treatment...
...,...,...,...,...,...,...,...
477,F-35: Closer Look at Colossally Costly Jet Tha...,Sputnik,2023-09-21,A US Marine Corps F-35B went down over South C...,WORLD,AMERICAS,Picture description not found
478,Toxic explosion in Nebraska shows US unable to...,Xu Yelu,2023-09-15,Barely two months after an explosion in Louisi...,WORLD,AMERICAS,A resident of East Palestine in the state of O...
479,Palestine sends rescue team to floods-hit east...,Xinhua,2023-09-14,\n\nA rescuer of a Palestinian rescue team pr...,PHOTO,WORLD,Rescuers of a Palestinian rescue team prepare ...
480,Armed clashes in Palestinian refugee camp inte...,Xinhua,2023-09-14,\n\nDisplaced Palestinian children wait to mo...,PHOTO,WORLD,Palestinians from Ain Al-Helweh camp prepare t...


In [45]:
#merge articles and authors
df =  articles.join(authors.set_index('author_name'), on='author', how='left')
#drop duplicates
df


,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description,author_title,author_description
0,Various celebrations held across world to cele...,Xinhua,2023-09-29,Musicians perform during a Mid-Autumn Festiva...,PHOTO,WORLD,Australian youths sing Chinese version of the ...,NaN,NaN
1,First Saudi ambassador to Palestine arrives in...,Xinhua,2023-09-27,\n\nSaudi ambassador to Palestine Naif bin Ba...,PHOTO,WORLD,Saudi ambassador to Palestine Naif bin Bandar ...,NaN,NaN
2,China releases a white paper on its proposals ...,GT staff reporters,2023-09-26,The Chinese government released a white paper...,CHINA,POLITICS,Photo: Li Xuanmin/GT,NaN,NaN
3,"Kaparot ceremony in Tel Aviv, Israel",Xinhua,2023-09-22,\n\nUltra Orthodox Jews perform the Kaparot c...,PHOTO,WORLD,Ultra Orthodox Jews perform the Kaparot ceremo...,NaN,NaN
4,"1,100 Gazan kidney patients facing drug shorta...",Xinhua,2023-09-21,\n\nA young Palestinian patient receives trea...,PHOTO,WORLD,A young Palestinian patient receives treatment...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
481,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
481,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
481,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
481,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."


In [13]:
#df = pd.merge(articles, authors, left_on='author', right_on='author_name', how='left')
#df = df.drop(columns=['author_name'])
#df.replace(pd.NA, 'Author detiles not found', inplace=True)
#remove duplicates
#df.to_csv('gaza_textcontain_before_new_preprocessed.csv')
df

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description,author_name,author_title,author_description
0,Various celebrations held across world to cele...,Xinhua,2023-09-29,Musicians perform during a Mid-Autumn Festiva...,PHOTO,WORLD,Australian youths sing Chinese version of the ...,NaN,NaN,NaN
1,First Saudi ambassador to Palestine arrives in...,Xinhua,2023-09-27,\n\nSaudi ambassador to Palestine Naif bin Ba...,PHOTO,WORLD,Saudi ambassador to Palestine Naif bin Bandar ...,NaN,NaN,NaN
2,China releases a white paper on its proposals ...,GT staff reporters,2023-09-26,The Chinese government released a white paper...,CHINA,POLITICS,Photo: Li Xuanmin/GT,NaN,NaN,NaN
3,"Kaparot ceremony in Tel Aviv, Israel",Xinhua,2023-09-22,\n\nUltra Orthodox Jews perform the Kaparot c...,PHOTO,WORLD,Ultra Orthodox Jews perform the Kaparot ceremo...,NaN,NaN,NaN
4,"1,100 Gazan kidney patients facing drug shorta...",Xinhua,2023-09-21,\n\nA young Palestinian patient receives trea...,PHOTO,WORLD,A young Palestinian patient receives treatment...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1753,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
1754,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
1755,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
1756,Greek scholar believes in ‘endless possibiliti...,Xu Liuliu,2023-09-14,"Chongqing, Sichuan, Shandong… Stelios Virvidak...",ARTS,CULTURE & LEISURE,The University of Athens Photo: VCG,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."


In [9]:
#senance count
df['sentence_count'] = df['article_text'].apply(lambda x: len(x.split('.')) - 1)

In [10]:
#word count using tokenization
#remove stopwords
stop_words = set(stopwords.words('english'))

In [11]:
#word count using tokenization
df['word_count'] = df['article_text'].apply(lambda x: len([word for word in word_tokenize(x) if word.isalpha()]))

In [12]:
# part of speech count using tokenization wihout stopwords
nltk.download('averaged_perceptron_tagger')
df['pos_count'] = df['article_text'].apply(lambda x: FreqDist([tag for word, tag in nltk.pos_tag(word_tokenize(x)) if word.isalpha() and word not in stop_words]))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


KeyboardInterrupt: 

In [ ]:
df['pos_count'].tolist()

In [ ]:
#pos count to data frame
df_pos = pd.DataFrame(df['pos_count'].tolist())
df_pos.fillna(0, inplace=True)
df_pos['sentence_count'] = df['sentence_count']
df_pos['word_count'] = df['word_count']
df_pos

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.graph_objects as go


In [ ]:
#word count; cut at 1000
plt.figure(figsize=(12, 10))
sns.histplot(df['word_count'], bins=20, kde=True)
plt.xlim(0, 1500)
#mean line
plt.axvline(df['word_count'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.legend()
plt.title('Word Count Distribution')
plt.show()

In [ ]:
#sentence count; cut at 100
plt.figure(figsize=(12, 10))
sns.histplot(df['sentence_count'], bins=20, kde=True)
plt.xlim(0,50)
#mean line
plt.axvline(df['sentence_count'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.legend()
plt.title('Sentence Count Distribution')
plt.show()


In [ ]:
#wordcloud form freqdist
fdist = FreqDist([word for word in word_tokenize(' '.join(df['article_text'])) if word.isalpha() and word not in stop_words])
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(fdist.most_common(100)))
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words')
plt.show()

In [ ]:
#top 10 most common part of speech
plt.figure(figsize=(12, 10))
df_pos.drop(columns=['sentence_count', 'word_count']).sum().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title('Top 10 Most Common Part of Speech')
#turn x labels
plt.xticks(rotation=45)
plt.show()

### Total POS

In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text']))) if tag =="RB" and word.isalpha() and word not in stop_words])

In [ ]:
plt.figure(figsize=(12, 18))
plt.subplot(3, 2, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(3, 2, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(3, 2, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(3, 2, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(3, 2, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(3, 2, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()


In [ ]:
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(fdist.most_common(100)))
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words')
plt.show()

In [ ]:
#sns countplot for article categories
plt.figure(figsize=(12, 10))
sns.countplot(y=df['article_category_one'], order=df['article_category_one'].value_counts().index, color='blue')
plt.title('Article Categories')


In [ ]:
#plot for category one = opinion
#3 plots in a row
plt.figure(figsize=(15, 10))

plt.subplot(1, 3, 1)
sns.countplot(x=df[df['article_category_one'] == 'OPINION']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'OPINION']['article_category_two'].value_counts().index )
plt.xticks(rotation=45)
plt.ylim(0, 600)
plt.title('OPINION Subcategories')

plt.subplot(1, 3, 2)
sns.countplot(x=df[df['article_category_one'] == 'WORLD']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'WORLD']['article_category_two'].value_counts().index)
plt.title('WORLD Subcategories')
plt.xticks(rotation=45)
plt.ylim(0, 600)

plt.subplot(1, 3, 3)
sns.countplot(x=df[df['article_category_one'] == 'CHINA']['article_category_two'],  color='blue', order=df[df['article_category_one'] == 'CHINA']['article_category_two'].value_counts().index)
plt.title('CHINA Subcategories')
plt.xticks(rotation=45)
plt.ylim(0, 600)
plt.show()


### POS: diplomacy vs columnists

In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'DIPLOMACY']))) if tag =="RB" and word.isalpha() and word not in stop_words])


In [ ]:
plt.figure(figsize=(6, 36))
plt.subplot(6, 1, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(6, 1, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(6, 1, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(6, 1, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(6, 1, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(6, 1, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()


In [ ]:
NN = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NN" and word.isalpha() and word not in stop_words])
NNP = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NNP" and word.isalpha() and word not in stop_words])
JJ = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="JJ" and word.isalpha() and word not in stop_words])
NNS = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="NNS" and word.isalpha() and word not in stop_words])
VB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="VB" and word.isalpha() and word not in stop_words])
RB = FreqDist([word for word, tag in nltk.pos_tag(word_tokenize(' '.join(df['article_text'][df['article_category_two'] == 'COLUMNISTS']))) if tag =="RB" and word.isalpha() and word not in stop_words])


In [ ]:
plt.figure(figsize=(6, 36))
plt.subplot(6, 1, 1)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NN')


plt.subplot(6, 1, 2)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNP.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNP')


plt.subplot(6, 1, 3)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(JJ.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of JJ')


plt.subplot(6, 1, 4)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NNS.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of NNS')


plt.subplot(6, 1, 5)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(RB.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of VB')

plt.subplot(6, 1, 6)
wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(dict(NN.most_common(100)))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Top 100 Most Common Words of RB')


plt.show()
